# Gauss-Quadrature

$$
\int_{a}^{b} \phi(t) dt = \sum_{i=1}^{m} \Phi(t_i) \alpha_i
$$

$$ 
\alpha_i = \int_{a}^{b} w(x) \prod_{j=1, j\neq i}^{m} \frac{t-t_j}{t_i-t_j} dt 
$$

## Gauss-Legendre Integration
- Integrate on $[-1,1]$
- $w(t) = 1$

$$
\int_{-1}^{1} f(x) dx = \sum_{i=1}^{m} f(x_i) \cdot \alpha_i
$$

Hereby the quadrature nodes are predefined.
See https://rosettacode.org/wiki/Numerical_integration/Gauss-Legendre_Quadrature#Python 
for reference.

## Variable limits
$$
\int_a^b f(t) dt = \frac{b-a}{2} \int_{-1}^{1} f\left(\frac{b-a}{2} x + \frac{a+b}{2}\right) dx = \frac{b-a}{2} \sum_{i=1}^{m} f\left(\frac{b-a}{2} x_i + \frac{a+b}{2}\right) \cdot \alpha_i
$$

In [31]:
import numpy as np

# Testfunction 
f = lambda x:  5.0 * (x-0.5) * np.exp(0.25 * (x-0.5)**2)
F = lambda x: 10.0 * np.exp(0.25 * (x-0.5)**2)


# Quadrature Nodes and Weights
t_i = [-np.sqrt(3./5.), 0.0, np.sqrt(3./5.)]
a_i = [5./9., 8./9., 5./9.]

'''
--------------------------
Integral over [-1,1]
--------------------------
'''
F_int_ana = F(1) - F(-1)
print("Analytic integral over [-1, 1]: %e" % (F_int_ana))

# Gauss-Legendre-Integration
F_int_GL = 0.0
for i in range(0,len(t_i)):
    F_int_GL += f(t_i[i]) * a_i[i]

print("Gauss-Legendre integral over [-1, 1]: %e" % (F_int_GL))


'''
--------------------------
Integral over [a,b]
--------------------------
'''
a = 3.315
b = 3.54
F_int_ana = F(b) - F(a)
print("Analytic integral over [%d, %d]: %e" % (a, b, F_int_ana))

# Gauss-Legendre-Integration
F_int_GL = 0.0
for i in range(0,len(t_i)):
    xi = ((b-a) / 2.0) * t_i[i] + (a+b)/2.0
    F_int_GL += f(xi) * a_i[i]
F_int_GL *= (b-a)/2.0
print("Gauss-Legendre integral over [-1, 1]: %e" % (F_int_GL))

Analytic integral over [-1, 1]: -6.905602e+00
Gauss-Legendre integral over [-1, 1]: -6.902713e+00
Analytic integral over [3, 3]: 2.828058e+01
Gauss-Legendre integral over [-1, 1]: 2.828058e+01


In [62]:
from numpy import *
 
##################################################################
# Recursive generation of the Legendre polynomial of order n
def Legendre(n,x):
    x=array(x)
    if (n==0):
        return x*0+1.0
    elif (n==1):
        return x
    else:
        return ((2.0*n-1.0)*x*Legendre(n-1,x)-(n-1)*Legendre(n-2,x))/n
    
    
##################################################################
# Derivative of the Legendre polynomials
def DLegendre(n,x):
    x=array(x)
    if (n==0):
        return x*0
    elif (n==1):
        return x*0+1.0
    else:
        return (n/(x**2-1.0))*(x*Legendre(n,x)-Legendre(n-1,x))
    
    
##################################################################
# Roots of the polynomial obtained using Newton-Raphson method
def LegendreRoots(polyorder,tolerance=1e-20):
    if polyorder<2:
        err=1 # bad polyorder no roots can be found
    else:
        roots=[]
        # The polynomials are alternately even and odd functions. So we evaluate only half the number of roots. 
        for i in range(1, int(int(polyorder) / 2) +1):
            x=cos(pi*(i-0.25)/(polyorder+0.5))
            error=10*tolerance
            iters=0
            while (error>tolerance) and (iters<1000):
                dx=-Legendre(polyorder,x)/DLegendre(polyorder,x)
                x=x+dx
                iters=iters+1
                error=abs(dx)
            roots.append(x)
        # Use symmetry to get the other roots
        roots=array(roots)
        if polyorder%2==0:
            roots=concatenate( (-1.0*roots, roots[::-1]) )
        else:
            roots=concatenate( (-1.0*roots, [0.0], roots[::-1]) )
        err=0 # successfully determined roots
    return [roots, err]


##################################################################
# Weight coefficients
def GaussLegendreWeights(polyorder):
    W=[]
    [xis,err]=LegendreRoots(polyorder)
    if err==0:
        W=2.0/( (1.0-xis**2)*(DLegendre(polyorder,xis)**2) )
        err=0
    else:
        err=1 # could not determine roots - so no weights
    return [W, xis, err]

##################################################################
# The integral value 
# func : the integrand
# a, b : lower and upper limits of the integral
# polyorder: order of the Legendre polynomial to be used
#
def GaussLegendreQuadrature(func, polyorder, a, b):
    [Ws,xs, err]= GaussLegendreWeights(polyorder)
    if err==0:
        ans=(b-a)*0.5*sum( Ws*func( (b-a)*0.5*xs+ (b+a)*0.5 ) )
    else: 
        # (in case of error)
        err=1
        ans=None
    return [ans,err]



##################################################################
# The integrand - change as required
def func(x):
    return exp(x)
##################################################################
#
order=5
a = 0
b = 2.0
[Ws,xs,err]=GaussLegendreWeights(order)
if err==0:
    print("Order    : ", order)
    print("Roots    : ", xs)
    print("Weights  : ", Ws)
else:
    print("Roots/Weights evaluation failed")
    
    
# Integrating the function
[ans,err]=GaussLegendreQuadrature(func , order, a, b)
if err==0:
    print("Integral : ", ans)
else:
    print("Integral evaluation failed")

Order    :  5
Roots    :  [-0.90617985 -0.53846931  0.          0.53846931  0.90617985]
Weights  :  [0.23692689 0.47862867 0.56888889 0.47862867 0.23692689]
Integral :  6.389056096688674


In [36]:
Legendre(3, [2,2,3,4])

array([ 17.,  17.,  63., 154.])